In [1]:
import numpy as np
import polars as pl
import pandas as pd
from sklearn.base import clone
from copy import deepcopy
import optuna
from scipy.optimize import minimize
import os
import matplotlib.pyplot as plt
import seaborn as sns

import re
from colorama import Fore, Style
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from IPython.display import clear_output
from concurrent.futures import ThreadPoolExecutor

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

import lightgbm as lgb
from catboost import CatBoostRegressor, CatBoostClassifier
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import *
from sklearn.metrics import *

# Preparing the data



Because some standard machine learning models cannot natively handle time series data, we have to transform time series data into a vector by extracting summary statistic to simplifies data representation and less computational complexity

In [2]:
def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet')) 
    # Read all parquet files that ends with part-0.parquet in series-train.parquet directory
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    # results is a tuple of statistic value with its id, like ([10.5, 20.3, 15.7], '001'),([12.1, 18.6, 14],'002')  
    stats, indexes = zip(*results)
    # now stats hold [10.5, 20.3, 15.7],... and indexes hold 001, 002,..
    df = pd.DataFrame(stats, columns=[f"Stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    
    return df

train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

100%|██████████| 2/2 [00:00<00:00, 12.53it/s]


In [3]:
time_series_cols = train_ts.columns.tolist()
time_series_cols.remove("id")

In [4]:
train = pd.merge(train, train_ts, how="left", on='id')
test = pd.merge(test, test_ts, how="left", on='id')
#We merge because train and train_ts, both of them have rows that correspond to a single child (with id)

In [5]:
# After merging we drop it cause we no longer need it
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

Dropping all the PCIAT-related columns because they are not in the test set

In [6]:
featuresCols = [
                'Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Season',
                'Fitness_Endurance-Max_Stage','Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 
                'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 
                'PAQ_C-Season',
                'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
                'PreInt_EduHx-computerinternet_hoursday', 'sii']

In [7]:
featuresCols += time_series_cols

In [8]:
train = train[featuresCols]

# Handling missing values

We are dropping features that have more than 80% missing values

In [9]:
threshold = 0.8
target = train['sii']
train = train[[col for col in train.columns if col != 'sii']].dropna(axis=1, thresh=(1 - threshold) * len(train))
test = test[train.columns] 
train['sii'] = target

# Removing Outliers

Replacing outliers in CGAS-CGAS_Score with NaN value

In [10]:
train[train['CGAS-CGAS_Score'] > 100]

,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,Stat_0,Stat_1,Stat_2,Stat_3,Stat_4,Stat_5,Stat_6,Stat_7,Stat_8,Stat_9,Stat_10,Stat_11,Stat_12,Stat_13,Stat_14,Stat_15,Stat_16,Stat_17,Stat_18,Stat_19,Stat_20,Stat_21,Stat_22,Stat_23,Stat_24,Stat_25,Stat_26,Stat_27,Stat_28,Stat_29,Stat_30,Stat_31,Stat_32,Stat_33,Stat_34,Stat_35,Stat_36,Stat_37,Stat_38,Stat_39,Stat_40,Stat_41,Stat_42,Stat_43,Stat_44,Stat_45,Stat_46,Stat_47,Stat_48,Stat_49,Stat_50,Stat_51,Stat_52,Stat_53,Stat_54,Stat_55,Stat_56,Stat_57,Stat_58,Stat_59,Stat_60,Stat_61,Stat_62,Stat_63,Stat_64,Stat_65,Stat_66,Stat_67,Stat_68,Stat_69,Stat_70,Stat_71,Stat_72,Stat_73,Stat_74,Stat_75,Stat_76,Stat_77,Stat_78,Stat_79,Stat_80,Stat_81,Stat_82,Stat_83,Stat_84,Stat_85,Stat_86,Stat_87,Stat_88,Stat_89,Stat_90,Stat_91,Stat_92,Stat_93,Stat_94,Stat_95,sii
2065,Fall,11,1,Winter,999.0,Fall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,Fall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,2.241,NaN,NaN,NaN,Fall,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
train.loc[train['CGAS-CGAS_Score'] == 999, 'CGAS-CGAS_Score'] = np.nan

Replacing zero stats in these physical features as NaN value

In [12]:
physical_cols = [
    'Physical-BMI', 'Physical-Height',
    'Physical-Weight', 'Physical-Waist_Circumference'
]
print((train[physical_cols] == 0).sum())

Physical-BMI                     7
Physical-Height                  0
Physical-Weight                 61
Physical-Waist_Circumference     0
dtype: int64


In [13]:
train[physical_cols] = train[physical_cols].replace(0, np.nan)
print((train[physical_cols] == 0).sum())

Physical-BMI                    0
Physical-Height                 0
Physical-Weight                 0
Physical-Waist_Circumference    0
dtype: int64


In [14]:
bp_cols = [
      'Physical-Diastolic_BP', 'Physical-Systolic_BP'
]
train[bp_cols] = train[bp_cols].replace(0, np.nan)
train.loc[train['Physical-Systolic_BP'] <= train['Physical-Diastolic_BP'], bp_cols] = np.nan

Replacing extreme outliers in BIA-related columns with NaN values.


In [15]:
bia_columns = [col for col in train.columns if 'BIA' in col]

cat_col = ['BIA-Season', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_Frame_num']
bia_columns = [col for col in bia_columns if col not in cat_col]
bia_columns

exclude_rows = []  # To store all indices to exclude

for col in bia_columns:
    Q1 = train[col].quantile(0.001)  # Lower quantile
    Q3 = train[col].quantile(0.999)  # Upper quantile

    # Replace with nan 
    train.loc[train[col] > Q3, col] = np.nan  # Replace above Q3 with NaN
    train.loc[train[col] < Q1, col] = np.nan  # Replace below Q1 with NaN
train

,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,Stat_0,Stat_1,Stat_2,Stat_3,Stat_4,Stat_5,Stat_6,Stat_7,Stat_8,Stat_9,Stat_10,Stat_11,Stat_12,Stat_13,Stat_14,Stat_15,Stat_16,Stat_17,Stat_18,Stat_19,Stat_20,Stat_21,Stat_22,Stat_23,Stat_24,Stat_25,Stat_26,Stat_27,Stat_28,Stat_29,Stat_30,Stat_31,Stat_32,Stat_33,Stat_34,Stat_35,Stat_36,Stat_37,Stat_38,Stat_39,Stat_40,Stat_41,Stat_42,Stat_43,Stat_44,Stat_45,Stat_46,Stat_47,Stat_48,Stat_49,Stat_50,Stat_51,Stat_52,Stat_53,Stat_54,Stat_55,Stat_56,Stat_57,Stat_58,Stat_59,Stat_60,Stat_61,Stat_62,Stat_63,Stat_64,Stat_65,Stat_66,Stat_67,Stat_68,Stat_69,Stat_70,Stat_71,Stat_72,Stat_73,Stat_74,Stat_75,Stat_76,Stat_77,Stat_78,Stat_79,Stat_80,Stat_81,Stat_82,Stat_83,Stat_84,Stat_85,Stat_86,Stat_87,Stat_88,Stat_89,Stat_90,Stat_91,Stat_92,Stat_93,Stat_94,Stat_95,sii
0,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,Fall,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,Fall,2.0,2.66855,16.8792,932.498,1492.00,8.25598,41.5862,13.8177,3.06143,9.21377,1.0,24.4349,8.89536,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,NaN,Fall,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,NaN,Fall,3.0,0.0,NaN,NaN,NaN,NaN,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,Winter,2.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.21172,3.97085,1.0,21.0352,14.97400,39.4497,15.4107,27.0552,Fall,2.340,Fall,46.0,64.0,Summer,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,NaN,65.0,94.0,117.0,Fall,Fall,20.0,1.0,10.2,1.0,14.7,2.0,7.0,1.0,10.0,1.0,10.0,1.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,NaN,60.0,97.0,117.0,Summer,Summer,18.0,1.0,NaN,NaN,NaN,NaN,5.0,0.0,7.0,0.0,7.0,0.0,7.0,1.0,Summer,3.0,3.84191,18.2943,1131.430,1923.44,15.59250,62.7757,14.0740,4.22033,18.82430,2.0,30.4041,16.77900,58.9338,26.4798,45.9966,Winter,2.451,Summer,31.0,45.0,Winter,0.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,-0

In [16]:
def feature_transformation(data):
    return data


In [17]:
# train = train[featuresCols]
train = train.dropna(subset='sii')
train

,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,Stat_0,Stat_1,Stat_2,Stat_3,Stat_4,Stat_5,Stat_6,Stat_7,Stat_8,Stat_9,Stat_10,Stat_11,Stat_12,Stat_13,Stat_14,Stat_15,Stat_16,Stat_17,Stat_18,Stat_19,Stat_20,Stat_21,Stat_22,Stat_23,Stat_24,Stat_25,Stat_26,Stat_27,Stat_28,Stat_29,Stat_30,Stat_31,Stat_32,Stat_33,Stat_34,Stat_35,Stat_36,Stat_37,Stat_38,Stat_39,Stat_40,Stat_41,Stat_42,Stat_43,Stat_44,Stat_45,Stat_46,Stat_47,Stat_48,Stat_49,Stat_50,Stat_51,Stat_52,Stat_53,Stat_54,Stat_55,Stat_56,Stat_57,Stat_58,Stat_59,Stat_60,Stat_61,Stat_62,Stat_63,Stat_64,Stat_65,Stat_66,Stat_67,Stat_68,Stat_69,Stat_70,Stat_71,Stat_72,Stat_73,Stat_74,Stat_75,Stat_76,Stat_77,Stat_78,Stat_79,Stat_80,Stat_81,Stat_82,Stat_83,Stat_84,Stat_85,Stat_86,Stat_87,Stat_88,Stat_89,Stat_90,Stat_91,Stat_92,Stat_93,Stat_94,Stat_95,sii
0,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,Fall,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,Fall,2.0,2.66855,16.8792,932.498,1492.00,8.25598,41.5862,13.8177,3.061430,9.21377,1.0,24.4349,8.89536,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,NaN,Fall,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,NaN,Fall,3.0,0.0,NaN,NaN,NaN,NaN,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,Winter,2.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.211720,3.97085,1.0,21.0352,14.97400,39.4497,15.4107,27.0552,Fall,2.340,Fall,46.0,64.0,Summer,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,NaN,65.0,94.0,117.0,Fall,Fall,20.0,1.0,10.2,1.0,14.7,2.0,7.0,1.0,10.0,1.0,10.0,1.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,NaN,60.0,97.0,117.0,Summer,Summer,18.0,1.0,NaN,NaN,NaN,NaN,5.0,0.0,7.0,0.0,7.0,0.0,7.0,1.0,Summer,3.0,3.84191,18.2943,1131.430,1923.44,15.59250,62.7757,14.0740,4.220330,18.82430,2.0,30.4041,16.77900,58.9338,26.4798,45.9966,Winter,2.451,Summer,31.0,45.0,Winter,0.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0

In [18]:
cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 'Fitness_Endurance-Season', 
          'FGC-Season', 'BIA-Season', 
         'PAQ_A-Season', 
         'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']
final_cat_columns = [col for col in cat_c if col in train.columns]

def update(df):
    for c in final_cat_columns: 
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
    return df

train = update(train)
test = update(test)


In [19]:
def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
# enumerate adds index to an iterable. unique_values = ['A','B','C'] => enumerate returns [(0, 'A'), (1, 'B'), (2, 'C')]
    return {value: idx for idx, value in enumerate(unique_values)}

# Similar to Label Encoder
for col in final_cat_columns:
    all_values = pd.concat([train[col], test[col]]).unique()
    mapping = {value: idx for idx, value in enumerate(all_values)}

    train[col] = train[col].replace(mapping).astype(int)
    test[col] = test[col].replace(mapping).astype(int)

In [20]:
# Dropping features have importance = 0 when training Light model based on different seeds

columns_drop = ['Stat_39', 'Stat_45', 'Stat_41', 'Stat_89', 'Stat_6', 'Stat_42', 
                 'Stat_7', 'Stat_10', 'Stat_9', 'Stat_11'
                , 'Stat_93', 'Stat_8'
               ]
train = train.drop(columns = columns_drop)
test = test.drop(columns = columns_drop)


In [21]:
SEED = 42
n_splits = 5

In [22]:
# Low importance features across different seeds
low_imp_features = []

In [23]:
# High importance features across different seeds
high_imp_features = []

In [24]:
%%time


#We need Confusion matrix, weight matrix and expected matrix
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')


def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))


# Optimization libraries like scipy.optimize.minimize work by minimizing a function. 
# Since QWK is a metric where higher is better, we negate it to allow the optimizer to "maximize" QWK indirectly.
def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

# TrainML, is a machine learning pipeline that performs model training, cross-validation, evaluation, 
# and test set prediction for a classification or regression task. 

def TrainML(model_class, test_data
            # , k_values=[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
           ):
    
    X = train.drop(['sii'], axis=1)
    y = train['sii']

    # best_kappa = -np.inf
    # best_k = None
    # best_model = None
    # best_submission = None

    # for n_splits in k_values:
    #     print(f"\nEvaluating for k = {n_splits} folds...")

    # Cross-validation, ensuring that the folds have the same distribution of target classes. 
    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    # Store QWK scores for training and validation sets.
    train_S = []
    test_S = []

    # predictions made by the model for the validation set during each fold of cross-validation.
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    # discrete predictions derived from oof_non_rounded using rounding logic
    oof_rounded = np.zeros(len(y), dtype=int) 
    # Stores the model's predictions for the test dataset during each fold of cross-validation
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        # train and test index are the index of this fold in the original training set
        # Data used for training in this fold
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        # Data used for validation in this fold
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]
        
        
        # Creates a new instance of an estimator with the same parameters as the original one 
        # but without any of the fitted data or state. Ensures each fold starts with a clean estimator.
        model = clone(model_class)
        model.fit(X_train, y_train)

        
        # Predict after fit to ensure model is trained correctly
        y_train_pred = model.predict(X_train)

        y_train_pred_rounded = y_train_pred.round(0).astype(int)
        
        y_val_pred = model.predict(X_val)

        # stores continuous data
        oof_non_rounded[test_idx] = y_val_pred
        # round the predicted value
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        # Calcute QWK for train and validation set
        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred_rounded)
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        # Stores QWK score each fold
        train_S.append(train_kappa)
        test_S.append(val_kappa)

        
        test_preds[:, fold] = model.predict(test_data)
        print(test_preds[:, fold])
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)


    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    #Minimize the kappa score, which find the best Threshold Rounder  
    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead') # Nelder-Mead | # Powell

    # If not converge, raise error
    assert KappaOPtimizer.success, "Optimization did not converge."

# rounded the oof_non_rounded which has a bunch of predicted continuous variables for the validation set at each fold
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

# Rounded the test predict variables with optimized ThresHold_Rounder
    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })

        # if tKappa > best_kappa:
        #     best_kappa = tKappa
        #     best_k = n_splits
        #     best_model = model
        #     best_submission = submission

    # print(f"Best k value: {best_k} with QWK Score: {best_kappa:.4f}")

    return submission,model

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.06 µs


# Hyperparameter Optimization using Optuna  

## Purpose of Hyperparameter Tuning  
To improve model performance, we used **Optuna** to optimize key hyperparameters for our model.  
The primary goal was to maximize the **QWK (Quadratic Weighted Kappa)** score while ensuring the model generalizes well without overfitting.

---

## Optuna Hyperparameter Tuning Process  

1. **Search Parameters**:  
   Optuna was configured to suggest optimal values for the following hyperparameters:  
   - **num_leaves**  
   - **max_depth**  
   - **learning_rate**  
   - **feature_fraction**  
   - **bagging_fraction**  
   - **bagging_freq**  
   - **lambda_l1** (L1 regularization)  
   - **lambda_l2** (L2 regularization)  
   - **min_data_in_leaf**  

2. **Cross-Validation Strategy**:  
   - **K-Fold Cross-Validation** was applied to split the data into multiple training and testing sets.  
   - The **average QWK score** across the folds was used to evaluate model performance.  

3. **Search Space Limitation**:  
   - Based on references from **Kaggle competitions** and iterative experiments, we adopted a **narrowed search space**.  
   - This ensures efficient exploration while focusing on high-performing parameter ranges.  
## Reason for Trials  
We limited the **number of trials to 50** for the following reasons:  
- **Efficient runtime**: Prevent excessively long computation times.  
- **Stable performance**: 50 trials provided consistent and optimal results without diminishing returns.  

---

## Result  
- With the optimized hyperparameters, the model achieved its **best QWK score**.  
- The **training score** was also reduced, minimizing overfitting and ensuring **better generalization** on unseen data.

### Optimal Hyperparameters  
After 50 trials (sufficient to balance time and performance), the following hyperparameters were identified as optimal:  

{
    'num_leaves': 484, 
    'max_depth': 11, 
    'learning_rate': 0.04533585929025977, 
    'feature_fraction': 0.8110477902071817, 
    'bagging_fraction': 0.7265461551046623, 
    'bagging_freq': 2, 
    'lambda_l1': 5.338437863405547, 
    'lambda_l2': 4.499492326361118e-06, 
    'min_data_in_leaf': 13
} data.  
2326361118e-06, 
    'min_data_in_leaf': 13
}


In [25]:
import optuna
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
import numpy as np

def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')


def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'mae',  
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 400, 500), 
        'max_depth': trial.suggest_int('max_depth', 10, 15),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.03, 0.05), 
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.7, 0.9),  
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.7, 0.9),  
        'bagging_freq': trial.suggest_int('bagging_freq', 2, 3),  
        'lambda_l1': trial.suggest_uniform('lambda_l1', 4, 6), 
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-6, 1e-4),  
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 15), 
    }

    X = train.drop(['sii'], axis=1)
    y = train['sii']

    n_splits = 5
    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    kappa_scores = []
    
    for train_idx, val_idx in SKF.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model = lgb.LGBMRegressor(**params, random_state=42)
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_val)
        y_pred_rounded = np.round(y_pred).astype(int)  # Làm tròn dự đoán cho QWK
        kappa = quadratic_weighted_kappa(y_val, y_pred_rounded)
        
        kappa_scores.append(kappa)
    
    # Trả về điểm số trung bình của QWK trên các fold
    return np.mean(kappa_scores)

In [26]:
study = optuna.create_study(direction='maximize')  
study.optimize(objective, n_trials=50)  

print("Best parameters:", study.best_params)
print("Best QWK score:", study.best_value)

[I 2024-12-18 14:32:00,875] A new study created in memory with name: no-name-844849d8-b232-4cbd-823e-35801470c26d


[LightGBM] [Warning] min_data_in_leaf is set=13, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=13
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.7227863256058192, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7227863256058192
[LightGBM] [Warning] lambda_l2 is set=2.870870283961034e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.870870283961034e-06
[LightGBM] [Warning] lambda_l1 is set=4.996715291910336, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.996715291910336
[LightGBM] [Warning] bagging_fraction is set=0.7769704370575458, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7769704370575458
[LightGBM] [Warning] min_data_in_leaf is set=13, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=13
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored

[I 2024-12-18 14:32:03,956] Trial 0 finished with value: 0.37271249619244584 and parameters: {'num_leaves': 494, 'max_depth': 12, 'learning_rate': 0.03203993350910327, 'feature_fraction': 0.7227863256058192, 'bagging_fraction': 0.7769704370575458, 'bagging_freq': 3, 'lambda_l1': 4.996715291910336, 'lambda_l2': 2.870870283961034e-06, 'min_data_in_leaf': 13}. Best is trial 0 with value: 0.37271249619244584.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] min_data_in_leaf is set=13, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=13
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.7227863256058192, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7227863256058192
[LightGBM] [Warning] lambda_l2 is set=2.870870283961034e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.870870283961034e-06
[LightGBM] [Warning] lambda_l1 is set=4.996715291910336, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.996715291910336
[LightGBM] [Warning] bagging_fraction is set=0.7769704370575458, subsample=1.0 will be ignored. Current value: bagging_f

[I 2024-12-18 14:32:07,988] Trial 1 finished with value: 0.3845275852077261 and parameters: {'num_leaves': 443, 'max_depth': 12, 'learning_rate': 0.04723993930644106, 'feature_fraction': 0.7284449408321994, 'bagging_fraction': 0.7658810922914697, 'bagging_freq': 3, 'lambda_l1': 4.1467269905334145, 'lambda_l2': 1.1306392744418763e-06, 'min_data_in_leaf': 13}. Best is trial 1 with value: 0.3845275852077261.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] min_

[I 2024-12-18 14:34:45,601] Trial 48 finished with value: 0.3797849231693233 and parameters: {'num_leaves': 451, 'max_depth': 10, 'learning_rate': 0.04488435035551296, 'feature_fraction': 0.792660008062601, 'bagging_fraction': 0.877205618367105, 'bagging_freq': 2, 'lambda_l1': 5.824683468025137, 'lambda_l2': 1.9490586253288278e-06, 'min_data_in_leaf': 12}. Best is trial 27 with value: 0.4031979760172506.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2024-12-18 14:34:48,951] Trial 49 finished with value: 0.40272366582353936 and parameters: {'num_leaves': 497, 'max_depth': 11, 'learning_rate': 0.04748707138438789, 'feature_fraction': 0.7753133273807322, 'bagging_fraction': 0.8522001537154474, 'bagging_freq': 2, 'lambda_l1': 5.796158884528135, 'lambda_l2': 5.578615501615373e-06, 'min_data_in_leaf': 14}. Best is trial 27 with value: 0.4031979760172506.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f